# Politics in Illinois Wesleyan

In [1]:
SCHOOL="illinoiswesleyan"
SUBJECT="opinions"
START_YEAR=2012
FINAL_YEAR=2023
DATA_DIR="data"      # should be 'data'
OUTPUT_DIR="output"  # should be 'output'

In [2]:
%pwd

'C:\\Users\\ranar\\ASDRP\\diversity-colleges\\notebooks'

In [3]:
%cd ..

C:\Users\ranar\ASDRP\diversity-colleges


In [4]:
import sys
sys.path.append('src')
import ouraws
import ourgraphs
import textutil

In [5]:
# NOTE: before loading, we need to be in the "diversity-colleges" folder

S3OBJECT_KEY = f"{DATA_DIR}/{SCHOOL}-{SUBJECT}-SNAPSHOT.parquet"
df = ouraws.getFromS3(S3OBJECT_KEY)

In [6]:
df.shape

(467, 6)

In [7]:
df.head(2)

,title,url,body,year,month,day
0,Horoscope: What spring break destination each ...,https://iwuargus.com/6017/opinions/horoscopes/...,Horoscope: What spring break destination each ...,2023,3,3
1,Opinion: Children’s show “Bluey” is enjoyable ...,https://iwuargus.com/6032/opinions/opinion-chi...,Opinion: Children’s show “Bluey” is enjoyable ...,2023,3,3


In [8]:
import requests
import os

BIPARTISAN_API_KEY = os.environ.get("BIPARTISAN_API_KEY")

BIPARTISAN_URL = "https://api.thebipartisanpress.com/api/endpoints/beta/robert"

In [9]:
BIPARTISAN_API_KEY

'gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g=='

In [10]:
df2022 = df[df['year'] == 2022]

articles_list = df2022['body'].to_list()
for article in articles_list:
    payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
    response = requests.post(BIPARTISAN_URL, data=payload)
    print(response.text)
    break

5.53969


In [11]:
df['year'].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2015, 2014, 2013, 2012],
      dtype=int64)

In [12]:
# SYNCHRONOUS METHOD : TAKES A LONG TIME

results = []

for year in range(2012, 2023):
    if year != 2016:
        df2 = df[df['year'] == year]
        print(f"Year: {year} ... {df2.shape[0]}")

        articles_list = df2['body'].to_list()
        value_sum = 0.0
        article_count = 0
        for article in articles_list:
            payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
            response = requests.post(BIPARTISAN_URL, data=payload)
            try:
                value_sum += float(response.text)
                article_count += 1
                # print(f"{article_count}", end=".")
            except:
                pass  # this is a non-number

        results.append({
            'year' : year,
            'article_count' : article_count,
            'polarity_sum' : value_sum,
            'polarity_avg' : float(value_sum) / article_count
        })

        print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
    # for each year, get all articles' bodies & send to bipartisan press API 
    #    and sum up the values & output the average for each year

Year: 2012 ... 33
2012	33	53.631942	1.6252103636363637
Year: 2013 ... 34
2013	34	152.948826	4.498494882352941
Year: 2014 ... 53
2014	53	39.851755999999995	0.7519199245283018
Year: 2015 ... 83
2015	83	293.488365	3.5360043975903612
Year: 2017 ... 2
2017	2	34.675653	17.3378265
Year: 2018 ... 15
2018	15	-89.67510399999999	-5.978340266666666
Year: 2019 ... 31
2019	31	-23.946132999999985	-0.7724559032258059
Year: 2020 ... 34
2020	34	-3.923991000000004	-0.11541150000000011
Year: 2021 ... 84
2021	84	-49.66045899999999	-0.5911959404761904
Year: 2022 ... 76
2022	76	167.045407	2.1979658815789476


In [ ]:
import pandas as pd

RESULTS_FILE = f"{DATA_DIR}/{SCHOOL}-POLARITY.parquet"

results_df = pd.DataFrame.from_records(results)
results_df.to_parquet(RESULTS_FILE)